In [21]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
OUTPUT_PATH="../data/yht_eur_big_table.csv"
INPUT_BAR_DATA_PATHS=["../data/EURUSD5_duka_2020.csv", "../data/EURUSD5_duka_2021.csv"]
INPUT_FM_RC_PATH="../data/yht_rc.csv"
SYMBOL="EURUSD"

In [23]:
from Utils.data_get_utils import get_duka_data_df_by_list
duka_5m_df = get_duka_data_df_by_list(INPUT_BAR_DATA_PATHS, SYMBOL)

In [24]:
duka_5m_df.head()

,symbol,datetime,open,high,low,close,volume
264,EURUSD,2020-01-01 22:00:00,1.12136,1.12139,1.12120,1.12125,47.60
265,EURUSD,2020-01-01 22:05:00,1.12125,1.12127,1.12125,1.12127,91.79
266,EURUSD,2020-01-01 22:10:00,1.12127,1.12127,1.12123,1.12127,64.98
267,EURUSD,2020-01-01 22:15:00,1.12127,1.12154,1.12127,1.12146,85.07
268,EURUSD,2020-01-01 22:20:00,1.12144,1.12153,1.12137,1.12146,123.93


In [25]:
from Utils.data_get_utils import get_fm_rc_data_df
from Utils.data_get_utils import calc_profit

rc_df = get_fm_rc_data_df(INPUT_FM_RC_PATH)
# current research is based on SYMBOL
rc_df=rc_df[rc_df["symbol"]==SYMBOL].reset_index(drop=True)
rc_df.head()

,symbol,action,lots,open_date,close_date,open_price,close_price,profits,pips,commission,swaps,profits_calc,commission_by_pips,date_var,swap_per_date_var_and_pip
0,EURUSD,buy,0.01,2021-08-11 07:16:00,2021-08-11 07:37:00,1.17093,1.17142,0.49,4.9,-0.04,0.0,0.45,-0.4,0,0.0
1,EURUSD,buy,0.01,2021-08-10 11:04:00,2021-08-10 18:55:00,1.17227,1.17219,-0.08,-0.8,-0.04,0.0,-0.12,-0.4,0,0.0
2,EURUSD,buy,0.01,2021-08-10 16:58:00,2021-08-10 18:02:00,1.17167,1.17194,0.27,2.7,-0.04,0.0,0.23,-0.4,0,0.0
3,EURUSD,buy,0.01,2021-08-10 16:02:00,2021-08-10 16:20:00,1.17165,1.17204,0.39,3.9,-0.04,0.0,0.35,-0.4,0,0.0
4,EURUSD,buy,0.01,2021-08-10 11:36:00,2021-08-10 15:24:00,1.17209,1.17241,0.32,3.2,-0.04,0.0,0.28,-0.4,0,0.0


In [26]:
rc_df.columns

Index(['symbol', 'action', 'lots', 'open_date', 'close_date', 'open_price',
       'close_price', 'profits', 'pips', 'commission', 'swaps', 'profits_calc',
       'commission_by_pips', 'date_var', 'swap_per_date_var_and_pip'],
      dtype='object')

In [27]:
from Utils.data_process_utils import get_period_bar_date
rc_df = get_period_bar_date(rc_df, ["open_date","close_date"], 5)
rc_df.head()

,symbol,action,lots,open_date,close_date,open_price,close_price,profits,pips,commission,swaps,profits_calc,commission_by_pips,date_var,swap_per_date_var_and_pip,open_date_mod_5,close_date_mod_5
0,EURUSD,buy,0.01,2021-08-11 07:16:00,2021-08-11 07:37:00,1.17093,1.17142,0.49,4.9,-0.04,0.0,0.45,-0.4,0,0.0,2021-08-11 07:15:00,2021-08-11 07:35:00
1,EURUSD,buy,0.01,2021-08-10 11:04:00,2021-08-10 18:55:00,1.17227,1.17219,-0.08,-0.8,-0.04,0.0,-0.12,-0.4,0,0.0,2021-08-10 11:00:00,2021-08-10 18:55:00
2,EURUSD,buy,0.01,2021-08-10 16:58:00,2021-08-10 18:02:00,1.17167,1.17194,0.27,2.7,-0.04,0.0,0.23,-0.4,0,0.0,2021-08-10 16:55:00,2021-08-10 18:00:00
3,EURUSD,buy,0.01,2021-08-10 16:02:00,2021-08-10 16:20:00,1.17165,1.17204,0.39,3.9,-0.04,0.0,0.35,-0.4,0,0.0,2021-08-10 16:00:00,2021-08-10 16:20:00
4,EURUSD,buy,0.01,2021-08-10 11:36:00,2021-08-10 15:24:00,1.17209,1.17241,0.32,3.2,-0.04,0.0,0.28,-0.4,0,0.0,2021-08-10 11:35:00,2021-08-10 15:20:00


### 检查是否 <交易记录数据> 是否都在 <行情Bar Data> 里面

In [28]:
%%time
# rc_df中的时间必须都在duka_5m_df当中存在
duka_5m_outer_join_rc_df = pd.merge(duka_5m_df, rc_df, left_on="datetime", right_on="open_date_mod_5", how="outer")
duka_5m_inner_join_rc_df = pd.merge(duka_5m_df, rc_df, left_on="datetime", right_on="open_date_mod_5", how="inner")
duka_5m_right_join_rc_df = pd.merge(duka_5m_df, rc_df, left_on="datetime", right_on="open_date_mod_5", how="right")
duka_5m_left_join_rc_df = pd.merge(duka_5m_df, rc_df, left_on="datetime", right_on="open_date_mod_5", how="left")
# duka_5m_left_join_rc_df = duka_5m_left_join_rc_df.dropna()

CPU times: user 42.1 ms, sys: 0 ns, total: 42.1 ms
Wall time: 41.9 ms


In [32]:
# classical methods to define the assert
print(len(duka_5m_left_join_rc_df.dropna()))  
print(len(rc_df))
assert len(duka_5m_left_join_rc_df.dropna()) == len(rc_df), (
        f'Some of the followme records are not in the instrument EURUSD input data.')

1890
1951


AssertionError: Some of the followme records are not in the instrument EURUSD input data.

### 把时间周期关系全部搞清出来

In [ ]:
# gmt + 8 == peking 
# gmt + 3 == tickmill

### 交易记录复现dataframe

In [ ]:
print("bar data length:", len(duka_5m_df))

In [ ]:
duka_5m_df.head()

In [ ]:
rc_df.head()

In [ ]:
duka_5m_left_join_rc_df = pd.merge(duka_5m_df, rc_df, 
                                   left_on="datetime", right_on="open_date_mod_5", 
                                   how="left")

### join 后得到的结果数量比原有的 <left_df: duka_5m_df> 多很正常
### 因为同一个位置有可能有多个单子

In [ ]:
# 保证所有followme当中出现的订单都在左表: duka_5m_df 中出现过
assert len(rc_df) == len(duka_5m_left_join_rc_df[duka_5m_left_join_rc_df["symbol_y"].notnull()]),(
    f'Some of the records data(rc_df) is lost, rc: ', 
    len(rc_df), " results: ", 
    len(duka_5m_left_join_rc_df[duka_5m_left_join_rc_df["symbol_y"].notnull()])
)

In [ ]:
%%time
from Utils.data_process_utils import slice_df_by_timerange
from datetime import timedelta

rc_start_dt = duka_5m_left_join_rc_df.dropna()["open_date"].min() - timedelta(hours=2)
rc_end_dt = duka_5m_left_join_rc_df.dropna()["close_date"].max() + timedelta(hours=2)
print("slicing date range : (", rc_start_dt, ", ", rc_end_dt, ")")
duka_5m_left_join_rc_date_filtered_df = slice_df_by_timerange(duka_5m_left_join_rc_df, "datetime", rc_start_dt, rc_end_dt)
analysis_df = duka_5m_left_join_rc_date_filtered_df.reset_index(drop=True)
analysis_df[analysis_df["symbol_y"].notnull()][["symbol_y","open_date_mod_5","close_date_mod_5"]].head()

In [ ]:
analysis_df.columns

### Calculates the current holding orders base on record data 

In [ ]:
%%time

def gen_dt_range(row):
    start_dt, end_dt = row["open_date_mod_5"], row["close_date_mod_5"]
    dt_range = (end_dt-start_dt)
    return dt_range

rc_df["dt_range"] = rc_df.apply(gen_dt_range, axis=1)
# Makes sure that the dt_range can be mod by 5
assert rc_df["dt_range"].apply(lambda x: x.seconds % (5*60) == 0).eq(True).all()

from datetime import datetime
from datetime import timedelta

def gen_dt_range_list(row):
    dt_range_list = []
    start_dt, end_dt = row["open_date_mod_5"], row["close_date_mod_5"]
    for dt_point in pd.date_range(start=start_dt, end=end_dt, freq='5min'):
        dt_range_list.append(dt_point.to_pydatetime())
    return dt_range_list

rc_df["dt_range_list"] = rc_df.apply(gen_dt_range_list, axis=1)
rc_df["dt_range_inter_num"] = rc_df["dt_range_list"].apply(lambda x: len(x))
rc_df["dt_range_inter_num"].sum()

In [ ]:
%%time
rc_explode_df = rc_df.explode('dt_range_list', ignore_index=True)
rc_explode_df = rc_explode_df.rename(columns={"dt_range_list": "dt_point_holding"})
# rc_explode_df.set_index(keys=["dt_point_holding"], drop=False, inplace=True)
# check_idx = 2
# print(rc_df.loc[check_idx,]["dt_range_list"])
# print(rc_df.loc[check_idx,]["open_date_mod_5"])
# print(rc_df.loc[check_idx,]["close_date_mod_5"])

In [ ]:
print(len(rc_explode_df))
rc_explode_df.head()

In [ ]:
%%time 
yht_rc_df_cnt = rc_explode_df.groupby(by=[rc_explode_df["dt_point_holding"]], dropna=False)\
                             .agg(cur_holding_cnt=("symbol", "count"))

In [ ]:
df_merged_l = pd.merge(rc_explode_df, yht_rc_df_cnt, left_on="dt_point_holding", right_on="dt_point_holding", 
                       how='left')
df_merged_o = pd.merge(rc_explode_df, yht_rc_df_cnt, left_on="dt_point_holding", right_on="dt_point_holding", 
                       how='outer')
assert len(df_merged_l) == len(df_merged_o), (
       f"left join results should be eq to the outer join"
)
rc_explode_df = df_merged_l
rc_explode_df.head()

In [ ]:
%%time
# save the records data to disk
rc_explode_df.set_index(keys=["dt_point_holding"], drop=False, inplace=True)
rc_explode_df.sort_index(axis=0, inplace=True)
rc_explode_df.to_csv(OUTPUT_PATH)